In [1]:
!pip install transformers bitsandbytes datasets peft gdown huggingface_hub trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.0 MB/s eta 0:00:00


In [2]:
import gdown

url = 'https://drive.google.com/file/d/1GJ0HyraB1DaUSrUUEgrez3F3CtgJFfat/view?usp=sharing'
output_path = '/kaggle/working/'
gdown.download(url, output_path, quiet=False,fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1GJ0HyraB1DaUSrUUEgrez3F3CtgJFfat
From (redirected): https://drive.google.com/uc?id=1GJ0HyraB1DaUSrUUEgrez3F3CtgJFfat&confirm=t&uuid=7a3b7d6c-6ef5-420e-80c2-0bf9e48f1226
To: /kaggle/working/Cleaned_Questions_Answers_For_Finetuning.csv
100%|██████████| 268M/268M [00:02<00:00, 118MB/s]  


'/kaggle/working/Cleaned_Questions_Answers_For_Finetuning.csv'

In [3]:
import torch
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer, SFTConfig, setup_chat_format
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [4]:
data = pd.read_csv('/kaggle/working/Cleaned_Questions_Answers_For_Finetuning.csv')

data = data.drop(columns=["Id", "ParentId"])

dataset = Dataset.from_pandas(data.iloc[:2000])

dataset

Dataset({
    features: ['Cleaned_Questions', 'Cleaned_Answers'],
    num_rows: 2000
})

In [21]:
model_name = "NousResearch/Hermes-3-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

lora_config = LoraConfig(
        r=16,
        lora_alpha=64,
        lora_dropout=0.05,
        bias="none",
        use_dora=True,
        task_type="CAUSAL_LM",
        target_modules=[
        "up_proj",
        "o_proj",
        "v_proj",
        "gate_proj",
        "q_proj",
        "down_proj",
        "k_proj"
      ])

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, lora_config)

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [6]:
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Cleaned_Questions"]},
                {"role": "assistant", "content": row["Cleaned_Answers"]}]
    
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    
    return row

tokenized_dataset = dataset.map(format_chat_template, num_proc=4)

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
training_args = SFTConfig(
    output_dir="./results",                        
    per_device_train_batch_size=8,           
    gradient_accumulation_steps=4,               
    eval_strategy="no",
    optim="adamw_torch",
    logging_strategy="steps",                  
    logging_steps=1,                        
    save_strategy="epoch",                       
    save_steps=500,                           
    save_total_limit=3,                         
    learning_rate=2e-4,                      
    num_train_epochs=6,
    lr_scheduler_type="linear",
    logging_dir="./logs",                    
    fp16=True,                               
    group_by_length=True,                       
    push_to_hub=False,                     
    report_to="tensorboard",
    dataloader_num_workers=4,
    overwrite_output_dir=True,          
    save_only_model=False,
    remove_unused_columns=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=lora_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_args,
    packing= False
)

trainer.train()

In [113]:
data['Cleaned_Questions'][548]

'How do I turn a python program into an .egg file?'

In [119]:
data['Cleaned_Answers'][548]

"Setuptools is the software that creates .egg files . It's an extension of the distutils package in the standard library. The process involves creating a setup.py file, then python setup.py bdist_egg creates an .egg package."

In [ ]:
messages = [
    {
        "role": "user",
        "content": "How do I turn a python program into an .egg file?"
    }
]

"""
answer before fine tuning:

"""

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, early_stopping=True, num_beams=5)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

question = messages[0]["content"]
answer = text.split("assistant")[1]

print(f"You: {question}")
print(f"Bot: {answer}")

In [ ]:
trainer.save_model("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

In [ ]:
from huggingface_hub import login, HfApi
import os

login("hf_....")

model_dir = "./fine_tuned_model"
checkpoint_dir = "./results/checkpoint-300"

api = HfApi()
repo_id = "thuan220401/Llama3_StackOverflow_FineTuning"

api.upload_folder(
    folder_path=model_dir, 
    repo_id=repo_id,
    repo_type="model"
)

api.upload_folder(
    folder_path=checkpoint_dir, 
    repo_id=repo_id,
    repo_type="model"
)